In [1]:
import utils

questoes = utils.load_json("./questoes/questoes.json")
text_questions = list(filter(lambda x : x["type"] == "only-text", questoes))
text_questions_str = list(map(lambda x : x['id'], text_questions))
text_models = ["phi4", "llava", "deepseek-r1", "llama3.2", "gemma2", "mistral"]


In [8]:
df = utils.new_test_table(
    questions=text_questions,
    models=text_models,
)
df.head(3)

,Model,Finish,OK,Null,Err,Tout,Acc,Ttot,Tle,Tavg,Tmax,Tmin
0,phi4,1608,1338,12,258,0,0.832090,10422.020824,0.0,6.481356,106.759996,0.320840
1,gemma2,1608,1208,30,370,0,0.751244,999.547631,0.0,0.621609,11.382335,0.271672
2,mistral,1608,931,30,647,0,0.578980,1477.893429,0.0,0.919088,20.847775,0.169479


In [13]:
def format_test_table(df):
    """Converte valores numéricos de tempo para string formatada na tabela."""
    df_copy = df.copy()
    for col in ["Ttot", "Tle", "Tavg", "Tmax", "Tmin"]:
        df_copy[col] = df_copy[col].apply(utils.format_time)
        
    df_copy["Finish"] = df_copy["Finish"].astype(str) + " (" + (df_copy["Finish"].astype(float) / df_copy["Finish"].max() * 100).round(0).astype(int).astype(str) + "%)"
    return df_copy

In [30]:
def format_test_table(df, total_questions=None):
    """Converte valores numéricos de tempo para string formatada na tabela."""
    df_copy = df.copy()
    for col in ["Ttot", "Tle", "Tavg", "Tmax", "Tmin"]:
        df_copy[col] = df_copy[col].apply(utils.format_time)
    
    if total_questions is None:
        total_questions = int(df["Finish"][:-1].max())
    
    percentage_function = lambda x: f"{x} ({100*x/total_questions:.1f}%)"
    finished_questions = int(df["Finish"].iloc[-1])
    
    df_copy["Finish"] = df_copy["Finish"].apply(percentage_function)
    df_copy.at[df_copy.index[-1], "Finish"] = f"{finished_questions} ({100*finished_questions/(total_questions*(len(df_copy)-1)):.1f}%)"

    return df_copy


In [31]:
format_test_table(df)

,Model,Finish,OK,Null,Err,Tout,Acc,Ttot,Tle,Tavg,Tmax,Tmin
0,phi4,1608 (100.0%),1338,12,258,0,0.832090,02:53:42,0.00,6.48,01:46,0.32
1,gemma2,1608 (100.0%),1208,30,370,0,0.751244,16:39,0.00,0.62,11.38,0.27
2,mistral,1608 (100.0%),931,30,647,0,0.578980,24:37,0.00,0.92,20.85,0.17
3,deepseek-r1,1608 (100.0%),884,126,598,0,0.549751,09:03:49,0.00,20.29,07:49,2.53
4,llava,1608 (100.0%),731,77,800,0,0.454602,15:13,0.00,0.57,10.90,0.15
5,llama3.2,1608 (100.0%),426,28,1154,0,0.264925,08:50,0.00,0.33,41.73,0.09
6,TOTAL,9648 (100.0%),5518,303,3827,0,0.571932,13:02:53,ND,4.87,07:49,0.09
